<a href="https://colab.research.google.com/github/adalbertii/modele-NLP/blob/main/word2vec_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Model wstepnie wytrenowany : gensim
- https://radimrehurek.com/gensim/

Wykorzystany w procesie uczenia modelu zbiór danych:  
- http://kavita-ganesan.com/entity-ranking-data/

In [1]:
import gzip
import gensim

In [3]:
# zamontować dysk Google

data_file="/content/drive/MyDrive/dane/word2vec/reviews_data.txt.gz"

with gzip.open (data_file, 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

In [4]:
print(line[:150])

b'Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did '


In [5]:
def read_input(input_file):
    """Metoda odczytuje plik wejściowy w formacie gzip"""

    print("odczyt pliku {0}...to może zająć trochę czasu".format(input_file))

    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f):

            if (i%10000==0):
                print ("odczytano {0} opinii".format (i))
            # wykonaj wstępne przetwarzanie i zwróć listę słów dla każdego tekstu recenzji
            yield gensim.utils.simple_preprocess (line)



# Instrukcja Yield wstrzymuje wykonywanie funkcji i wysyła wartość z powrotem do funkcji wywołującej, ale zachowuje stan,
# aby umożliwić wznowienie funkcji od miejsca, w którym została przerwana.
# Po wznowieniu działania funkcja kontynuuje wykonywanie
# Dzięki temu kod może generować w czasie serię wartości, zamiast wyznaczać je na raz i wysyłać z powrotem w formie listy.

In [6]:
# odczyt stokenizowanych opinii do listy
# każdy element opinii  staje się serią słów
#, więc budujemy listę list

documents = list (read_input (data_file))
print ("Odczyt danych  z pliku zakńczony ")

odczyt pliku /content/drive/MyDrive/dane/word2vec/reviews_data.txt.gz...to może zająć trochę czasu
odczytano 0 opinii
odczytano 10000 opinii
odczytano 20000 opinii
odczytano 30000 opinii
odczytano 40000 opinii
odczytano 50000 opinii
odczytano 60000 opinii
odczytano 70000 opinii
odczytano 80000 opinii
odczytano 90000 opinii
odczytano 100000 opinii
odczytano 110000 opinii
odczytano 120000 opinii
odczytano 130000 opinii
odczytano 140000 opinii
odczytano 150000 opinii
odczytano 160000 opinii
odczytano 170000 opinii
odczytano 180000 opinii
odczytano 190000 opinii
odczytano 200000 opinii
odczytano 210000 opinii
odczytano 220000 opinii
odczytano 230000 opinii
odczytano 240000 opinii
odczytano 250000 opinii
Odczyt danych  z pliku zakńczony 


In [ ]:
# budowa modelu
# przy zainicjowanym T4 GPU procesorze zajmuje to około 30 minut

model = gensim.models.Word2Vec(documents,
                               window=10,   # Maksymalna odległość między bieżącym a przewidywanym słowem w zdaniu
                               min_count=2, # Ignoruje wszystkie słowa o całkowitej częstotliwości mniejszej niż ta
                               workers=10)  # ile wątków roboczych do szkolenia modelu
                                            # (szybsze szkolenie na maszynach wielordzeniowych).

model.train(documents,total_examples=len(documents),epochs=10)

In [ ]:
model.save('/content/drive/MyDrive/dane/modele/mymodel-wor2vec')

**Załadowanie wcześniej wytrenowanego modelu**

In [13]:
new_model = gensim.models.Word2Vec.load('/content/drive/MyDrive/dane/modele/mymodel-wor2vec')

In [ ]:
# jaka jest liczba wyrazów w modelu (korpusie modelu)
new_model.corpus_total_words

In [14]:
# wyszukiwania słów podobnych do słowa 'dirty;
# Spowoduje to zwrócenie 10 najpopularniejszych, podobnych słów.

w1 = "dirty"
new_model.wv.most_similar (positive=w1)

[('filthy', 0.8984195590019226),
 ('grubby', 0.8221594095230103),
 ('smelly', 0.8083983063697815),
 ('unclean', 0.807628870010376),
 ('stained', 0.8064349293708801),
 ('dusty', 0.800523042678833),
 ('dingy', 0.7889810800552368),
 ('gross', 0.7759803533554077),
 ('mouldy', 0.7633497714996338),
 ('soiled', 0.7610918879508972)]

In [18]:
# wyszukanie 6 podobnych słów do słowa 'polite'

w1 = ["polite"]
new_model.wv.most_similar (positive=w1,topn=6)

[('courteous', 0.9318776726722717),
 ('friendly', 0.8612350225448608),
 ('cordial', 0.8443536758422852),
 ('curteous', 0.828039824962616),
 ('professional', 0.8158765435218811),
 ('attentive', 0.8073211312294006)]

In [19]:
# # wyszukanie 6 podobnych słów do słowa 'france'
w1 = ["france"]
new_model.wv.most_similar (positive=w1,topn=6)


[('germany', 0.7637171745300293),
 ('spain', 0.7122423052787781),
 ('hawaii', 0.7105596661567688),
 ('russia', 0.710435688495636),
 ('canada', 0.6956576108932495),
 ('austria', 0.6877645254135132)]

In [21]:
# wyszukanie 6 podobnych słów do słowa 'shocked'
w1 = ["shocked"]
new_model.wv.most_similar (positive=w1,topn=6)

[('horrified', 0.8368116617202759),
 ('amazed', 0.8187119960784912),
 ('dismayed', 0.7783960700035095),
 ('appalled', 0.7749536633491516),
 ('astounded', 0.7728244662284851),
 ('astonished', 0.7671551704406738)]

In [ ]:
# wyznaczenie podobieństwa dwóch słów (wyrażonego liczbowo!!!)
new_model.wv.similarity(w1="dirty",w2="smelly")


0.8083983

In [23]:
# podobieństwo poiędzy dwoma identycznymi wyrazami
new_model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [24]:
# similarity between two unrelated words
new_model.wv.similarity(w1="dirty",w2="clean")

0.28794798

**Testowanie gotowego modelu "glove-twitter-25"**

Podstawowy zbiór danych: Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased)

    https://nlp.stanford.edu/projects/glove/
    https://nlp.stanford.edu/pubs/glove.pdf



In [26]:
import gensim.downloader as api

In [27]:
model_glove = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [28]:
model_glove.most_similar("cat", topn =10)

[('dog', 0.9590820074081421),
 ('monkey', 0.920357882976532),
 ('bear', 0.9143136739730835),
 ('pet', 0.9108031392097473),
 ('girl', 0.8880629539489746),
 ('horse', 0.8872726559638977),
 ('kitty', 0.8870542049407959),
 ('puppy', 0.886769711971283),
 ('hot', 0.886525571346283),
 ('lady', 0.8845519423484802)]

In [29]:
model_glove.similarity('cat','dog')

0.95908207

In [31]:
#model_glove.word_vec('cat')
model_glove.get_vector('cat')

array([-0.96419 , -0.60978 ,  0.67449 ,  0.35113 ,  0.41317 , -0.21241 ,
        1.3796  ,  0.12854 ,  0.31567 ,  0.66325 ,  0.3391  , -0.18934 ,
       -3.325   , -1.1491  , -0.4129  ,  0.2195  ,  0.8706  , -0.50616 ,
       -0.12781 , -0.066965,  0.065761,  0.43927 ,  0.1758  , -0.56058 ,
        0.13529 ], dtype=float32)

In [32]:
model_glove.get_vector('dog')

array([-1.2420e+00, -3.5980e-01,  5.7285e-01,  3.6675e-01,  6.0021e-01,
       -1.8898e-01,  1.2729e+00, -3.6921e-01,  8.9080e-02,  4.0339e-01,
        2.5130e-01, -2.5548e-01, -3.9209e+00, -1.1100e+00, -2.1308e-01,
       -2.3846e-01,  9.5322e-01, -5.2750e-01, -7.8049e-04, -3.5771e-01,
        5.5582e-01,  7.7869e-01,  4.6874e-01, -7.7803e-01,  7.8378e-01],
      dtype=float32)

In [33]:
#który wyraz nie pasuje?
model_glove.doesnt_match(["trump","bernie","obama","pelosi","orange"])

'orange'

**Trenowanie modelu na bazie załadowanego korpusu**

In [45]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import warnings
warnings.filterwarnings('ignore')

In [46]:
# załądowanie korpusu  'text8'
# Pierwsze 100 000 000 bajtów zwykłego tekstu z Wikipedii. Używane do celów testowych;

# http://mattmahoney.net/dc/textdata.html
# https://github.com/piskvorky/gensim-data/blob/master/README.md




corpus = api.load('text8')

In [47]:
# trenowanie modelu  Word2Vec
# uczenie trwa prawie 3 minuty (przy aktywnym procesorze T4 GPU)

model_text8 = Word2Vec(corpus,
                       window=10,   # Maksymalna odległość między bieżącym a przewidywanym słowem w zdaniu
                       min_count=2, # Ignoruje wszystkie słowa o całkowitej częstotliwości mniejszej niż 2
                       workers=10)  # ile wątków roboczych do szkolenia modelu
                                    #( szybsze szkolenie na maszynach wielordzeniowych).


In [48]:
# wyrazy podobne
model_text8.wv.most_similar("shocked")

[('outraged', 0.8016893267631531),
 ('betrayed', 0.7413463592529297),
 ('upset', 0.6976285576820374),
 ('surprised', 0.6957406401634216),
 ('greeted', 0.6946792602539062),
 ('told', 0.6861076951026917),
 ('avenged', 0.6719164848327637),
 ('disappointed', 0.665448784828186),
 ('stabbed', 0.6623957753181458),
 ('beaten', 0.651833713054657)]

In [49]:
# podobieństwo pomiędzy dwoma wyrazami
model_text8.wv.similarity(w1="dirty",w2="smelly")

0.39632335

In [50]:
# Który z nich nie pasuje do pozostałych na liście ?
model_text8.wv.doesnt_match(["cat","dog","france"])

'france'